# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [5]:
nutrition_agent  = Agent(
    name= "nutrition assistant",
    instructions= """
    you are a helpful assistant giving out nutrition advice
    your communication style is professional
    you are polite and caring
    you will give concise answers
    you will reply don't know if you do not know the answer to a question
    you will avoid violence , gambling, pornography related subjects and sanitize questions and answers accordingly.  
"""
)

Let's execute the Agent:

In [6]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="nutrition assistant", ...)
- Final output (str):
    Overall, bananas are a healthy, nutrient-dense fruit. Key points:
    
    - Nutrients: good source of potassium, vitamin B6, vitamin C, and dietary fiber.
    - Calories: about 90–110 kcal for a medium banana.
    - Benefits: supports heart health, digestion, and provides steady natural sugars for quick energy.
    - Considerations: higher in sugars and carbs than some fruits, so portion if you’re managing blood sugar; ripe bananas are sweeter (higher sugar) than less-ripe.
    
    Tips: pair with protein (e.g., peanut butter) for fullness, or slice into yogurt or oats. Generally fine for most people in moderate portions.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [7]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are a healthy, convenient fruit.

Key positives:
- Rich in potassium (supports heart and blood pressure), vitamin B6, vitamin C, and dietary fiber.
- Moderate in natural sugars but low in fat; provide lasting energy.
- Generally inexpensive and portable.

Things to consider:
- If you monitor sugar or calories closely (e.g., diabetes management or weight loss), portion size matters. A medium banana (~105 calories) is a common serving.
- Ripeness affects sugar content; riper bananas are sweeter and contain more available sugars.
- They are easy to include in a balanced diet but vary other fruits to ensure a range of nutrients.

Typical guidance:
- 1 medium banana counts as 1 of your daily fruit servings. Most adults aim for about 2 cups of fruit per day (varies by guidelines).

_Good Job!_